In [2]:
# LIBRARIES TO BE USED: 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import re
import operator
from collections import Counter
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix,hstack
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
#Plot styling
import seaborn as sns; sns.set()  # for plot styling
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

%matplotlib inline


/Users/Naif/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
!pip install nltk  # Install natrual language toolkit

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import nltk; nltk.download()  
# If GUI does not show up: d, Enter, stopwords, Enter, q
# If GUI does show up: Corpa > double click Stopwords

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

#### Loading training dataset for task-1

In [331]:
# TRAIN DATA 
data1=pd.read_csv('MADAR-Corpus-26-train.tsv', sep='\t', header=-1)
data2=pd.read_csv('MADAR-Corpus-6-train.tsv', sep='\t', header=-1)
datadev2=pd.read_csv('MADAR-Corpus-6-dev.tsv', sep='\t', header=-1)
data1=data1.rename(columns={0: 'text', 1: 'category'})
data2=data2.rename(columns={0: 'text', 1: 'category'})
datadev2=datadev2.rename(columns={0: 'text', 1: 'category'})
data_task1=pd.concat([data1,data2,datadev2])
print(data1.shape)
print(data2.shape)
print(data_task1.shape)
print("number of different categories: " + str(len(set(data_task1.category.tolist()))))
data_task1.head()

(41600, 2)
(54000, 2)
(101600, 2)
number of different categories: 26


,text,category
0,هناك ، أمام بيانات السائح تماما .,MSA
1,لم اسمع بهذا العنوان من قبل بالقرب من هنا .,MSA
2,استمر في السير في هذا الطريق حتى تجد صيدلية .,MSA
3,كم تكلفة الإفطار ؟,MSA
4,كيف أستطيع مساعدتك ؟,MSA


#### Loading test dataset for task-1

In [332]:
# TEST DATA 

datadev1=pd.read_csv('MADAR-Corpus-26-dev.tsv', sep='\t', header=-1)
# datadev2=pd.read_csv('MADAR-Corpus-6-dev.tsv', sep='\t', header=-1)
datadev1=datadev1.rename(columns={0: 'text', 1: 'category'})
# datadev2=datadev2.rename(columns={0: 'text', 1: 'category'})
data_dev1=datadev1
# print(datadev1.shape)
# print(datadev2.shape)
print(data_dev1.shape)
print("number of different categories: " + str(len(set(data_dev1.category.tolist()))))
data_dev1.head()

(5200, 2)
number of different categories: 26


,text,category
0,بالمناسبة ، اسمي هيروش إيجيما .,MSA
1,"هذا القطار يتوقف في لاك فورست , أليس كذلك ؟",MSA
2,"هذا الكارت , حسناً ؟",MSA
3,لم يخرج من الماكينة شيء .,MSA
4,عندك أية شيء يمكن أن أتعاطه للطفح الجلدي ؟,MSA


### Task 2 dataset

In [333]:
# DEV # 
# data_twitter=pd.read_csv('new_dataset_DEV_tweets_features.csv', sep='\t', header=0)
# aa=data_twitter.iloc[:,3].str.split(',', expand=True)
# aa.columns = ['ALE','ALG','ALX','AMM','ASW','BAG','BAS','BEI','BEN','CAI','DAM','DOH','FES','JED','JER','KHA','MOS','MSA','MUS','RAB','RIY','SAL','SAN','SFX','TRI','TUN']
# data_twitter=data_twitter.drop(data_twitter.columns[3], axis=1)
# data_twitter=data_twitter.join(aa)
# data_twitter.columns = ["user", "tweet_ID", "language", "user_country", "tweet_text", 'ALE','ALG','ALX','AMM','ASW','BAG','BAS','BEI','BEN','CAI','DAM','DOH','FES','JED','JER','KHA','MOS','MSA','MUS','RAB','RIY','SAL','SAN','SFX','TRI','TUN']
# data_twitter.to_csv('final_new_dataset_DEV_tweets_features.csv')
# data_twitter
twitter_data=pd.read_csv('final_new_dataset_DEV_tweets_features.csv')
twitter_data=twitter_data.drop(twitter_data.columns[0], axis=1)
print(twitter_data.shape)
twitter_data.head()


(29869, 31)


,user,tweet_ID,language,user_country,tweet_text,ALE,ALG,ALX,AMM,ASW,...,MOS,MSA,MUS,RAB,RIY,SAL,SAN,SFX,TRI,TUN
0,qtri111,1094562990404354048,ar,Qatar,RT @marzoqi_w: @AhmadQatar تواصلت مع إدارة الص...,0.0011,0.0001,0.0001,0.1475,0.0001,...,0.0,0.0005,0.0225,0.0,0.0306,0.0476,0.0002,0.0003,0.0005,0.0007
1,qtri111,1094559081078157313,ar,Qatar,RT @marzoqi_w: القول بان ينبغي عدم اثارة القضا...,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.9715,0.0013,0.0,0.0006,0.0000,0.0265,0.0000,0.0000,0.0000
2,qtri111,1094552522855976960,ar,Qatar,RT @MUTARED: عندما يسجن العلماء والمفكرون والد...,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0155,0.9843,0.0,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000
3,qtri111,1094552403062542336,ar,Qatar,RT @hfc_am1: ماقدرت أكمل الفيديو من اللي فيه !...,0.0012,0.0000,0.0001,0.0338,0.0010,...,0.0,0.0000,0.8100,0.0,0.0895,0.0179,0.0002,0.0000,0.0000,0.0000
4,qtri111,1094551366599667712,ar,Qatar,RT @HDMshk: هاتان الدقيقتان ينبغي أن تدرس في ا...,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.9999,0.0001,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [334]:
# data_twitter2=pd.read_csv('new_dataset_TRAIN_tweets_features.csv', sep='\t', header=0)
# aa=data_twitter2.iloc[:,3].str.split(',', expand=True)
# aa.columns = ['ALE','ALG','ALX','AMM','ASW','BAG','BAS','BEI','BEN','CAI','DAM','DOH','FES','JED','JER','KHA','MOS','MSA','MUS','RAB','RIY','SAL','SAN','SFX','TRI','TUN']
# data_twitter2=data_twitter2.drop(data_twitter2.columns[3], axis=1)
# data_twitter2=data_twitter2.join(aa)
# data_twitter2.columns = ["user", "tweet_ID", "language", "user_country", "tweet_text", 'ALE','ALG','ALX','AMM','ASW','BAG','BAS','BEI','BEN','CAI','DAM','DOH','FES','JED','JER','KHA','MOS','MSA','MUS','RAB','RIY','SAL','SAN','SFX','TRI','TUN']
# data_twitter2.to_csv('final_new_dataset_TRAIN_tweets_features.csv')
# data_twitter2

twitter_data2=pd.read_csv('final_new_dataset_TRAIN_tweets_features_wRETWEETnum.csv')
twitter_data2=twitter_data2.drop(twitter_data2.columns[0], axis=1)
print(twitter_data2.shape)
data=twitter_data2
data.head()


(217592, 32)


,user,tweet_ID,language,user_country,tweet_text,ALE,ALG,ALX,AMM,ASW,...,MSA,MUS,RAB,RIY,SAL,SAN,SFX,TRI,TUN,is_retweet
0,uJ4DPgC8LZxsFJ6,1094598211673690112,ar,Oman,RT @waha1950: (أَمْسَيْنَا وَأَمْسَى الْمُلْكُ...,0.0000,0.0000,0.0,0.0000,0.0000,...,0.0002,0.9976,0.0,0.0010,0.0001,0.0001,0.0000,0.0000,0.0000,1
1,uJ4DPgC8LZxsFJ6,1094598132720193543,ar,Oman,RT @Mr_smmsm: اللهم إنى أعوذ بك من الهم و الحز...,0.0000,0.9648,0.0,0.0001,0.0000,...,0.0009,0.0034,0.0,0.0075,0.0025,0.0002,0.0046,0.0015,0.0088,1
2,uJ4DPgC8LZxsFJ6,1094597914985459713,ar,Oman,RT @waha1950: (مَن قال حين يُمسِي ثلاث مرات أَ...,0.0000,0.0000,0.0,0.0001,0.0000,...,0.0205,0.4342,0.0,0.0305,0.0013,0.0016,0.0000,0.0001,0.0000,1
3,uJ4DPgC8LZxsFJ6,1094597887185637376,ar,Oman,RT @90_cutte: لا إله إلا أنت سبحانك إني كنت من...,0.0015,0.0000,0.0,0.0079,0.0106,...,0.0235,0.0099,0.0,0.0001,0.0002,0.0000,0.0000,0.0000,0.0000,1
4,uJ4DPgC8LZxsFJ6,1094597862879580162,ar,Oman,RT @NF7AT11: قال ﷺ: من صلى عليَّ صلاة واحدة صل...,0.0000,0.0000,0.0,0.0000,0.0000,...,0.9930,0.0063,0.0,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,1


In [6]:
user_label=pd.read_csv('MADAR-Twitter-Subtask-2.TRAIN.user-label.tsv', sep='\t', header=0)
print(user_label.shape)
user_label.head() #seems useless since it is already in the data 

(2180, 2)


,#1 User,#2 Label
0,uJ4DPgC8LZxsFJ6,Oman
1,NasrMoza,Qatar
2,RandaHabib,Jordan
3,h1kFsEDzLswl4zM,Qatar
4,hu_dly,Iraq


#### Preprocessing pipeline

In [6]:
from nltk.corpus import stopwords
stops = set(stopwords.words('Arabic'))
noise=list(stops)
print(noise)

['هيا', 'لوما', 'لو', 'هي', 'نحو', 'كيف', 'إليكم', 'أنتن', 'أو', 'بك', 'بمن', 'تين', 'كلاهما', 'له', 'هنا', 'ومن', 'والذين', 'أولاء', 'بس', 'لي', 'هن', 'إليك', 'تي', 'إما', 'إنما', 'آي', 'لسن', 'منه', 'هيهات', 'هو', 'أف', 'لكيلا', 'إذ', 'بخ', 'ليت', 'مه', 'هذه', 'أنت', 'بل', 'هؤلاء', 'التي', 'ذلكما', 'كما', 'لعل', 'أوه', 'قد', 'ذينك', 'أولئك', 'لست', 'وإذا', 'لسنا', 'هذي', 'أما', 'آها', 'سوف', 'مهما', 'من', 'لهما', 'لن', 'بها', 'ذلك', 'ذا', 'لنا', 'منها', 'ها', 'ألا', 'إن', 'بيد', 'هذان', 'ذوا', 'اللاتي', 'عل', 'ليست', 'أم', 'الذين', 'مذ', 'فإن', 'خلا', 'لولا', 'عليه', 'ليس', 'به', 'منذ', 'بماذا', 'بعد', 'شتان', 'إلى', 'إذن', 'بلى', 'إليكن', 'بكما', 'ذي', 'على', 'لما', 'هيت', 'كم', 'بعض', 'ثم', 'هاتي', 'لستن', 'وما', 'أنا', 'فمن', 'بكن', 'كلتا', 'نحن', 'ولو', 'لكنما', 'لم', 'لستما', 'فيه', 'اللواتي', 'وإن', 'وهو', 'ليسوا', 'هاتين', 'عدا', 'كأي', 'بي', 'ذواتي', 'كيت', 'لستم', 'مع', 'حبذا', 'إليكما', 'إنه', 'نعم', 'تينك', 'حيثما', 'متى', 'آه', 'دون', 'ته', 'ذان', 'هما', 'لهم', 'إي', 'وال

In [335]:
# def remove_duplicate(d):
#     d = d.split(" ")
#     UniqW = Counter(d) 
#     s = " ".join(UniqW.keys()) 
#     return s

noise_list = ['.','!','،','؟'] #punctuation 
def remove_noise(input_text):
    words = input_text.split() 
    noise_free_words = [word for word in words if word not in noise_list] 
    noise_free_text = " ".join(noise_free_words) 
    return noise_free_text

def remove_english(a): #remove any english word 
    c=[]
    for string in a.split():
        if not re.search(r'[a-zA-Z]', string):
            c.append(string)
    aa=' '.join(c)
    return aa 

def char_gram(b,n=5): #charecter level ngrams 
    last=[]
    for n in range(2,n+1,1):
        x[n]=[b[i:i+n] for i in range(len(b)-n+1)]
        last+=x[n]
    last+=b.split()
    return "<some_space>".join(last)

#char_gram calculates all ngrams from n=2 to the specified n. 

In [336]:
#example of char_gram  
b='أمام بيانات السائح تماما'
char_gram(b)

'أم<some_space>ما<some_space>ام<some_space>م <some_space> ب<some_space>بي<some_space>يا<some_space>ان<some_space>نا<some_space>ات<some_space>ت <some_space> ا<some_space>ال<some_space>لس<some_space>سا<some_space>ائ<some_space>ئح<some_space>ح <some_space> ت<some_space>تم<some_space>ما<some_space>ام<some_space>ما<some_space>أما<some_space>مام<some_space>ام <some_space>م ب<some_space> بي<some_space>بيا<some_space>يان<some_space>انا<some_space>نات<some_space>ات <some_space>ت ا<some_space> ال<some_space>الس<some_space>لسا<some_space>سائ<some_space>ائح<some_space>ئح <some_space>ح ت<some_space> تم<some_space>تما<some_space>مام<some_space>اما<some_space>أمام<some_space>مام <some_space>ام ب<some_space>م بي<some_space> بيا<some_space>بيان<some_space>يانا<some_space>انات<some_space>نات <some_space>ات ا<some_space>ت ال<some_space> الس<some_space>السا<some_space>لسائ<some_space>سائح<some_space>ائح <some_space>ئح ت<some_space>ح تم<some_space> تما<some_space>تمام<some_space>ماما<some_space>أمام <some_

In [337]:
# apply the three preprocessing functions to the training dataset

data_task1['text']=data_task1['text'].apply(remove_noise)
data_task1['text']=data_task1['text'].apply(remove_english)
data_task1['text']=data_task1['text'].apply(char_gram)

data_task1.head()

,text,category
0,هن<some_space>نا<some_space>اك<some_space>ك <s...,MSA
1,لم<some_space>م <some_space> ا<some_space>اس<s...,MSA
2,اس<some_space>ست<some_space>تم<some_space>مر<s...,MSA
3,كم<some_space>م <some_space> ت<some_space>تك<s...,MSA
4,كي<some_space>يف<some_space>ف <some_space> أ<s...,MSA


In [338]:
# apply the three preprocessing functions to the test dataset

data_dev1['text']=data_dev1['text'].apply(remove_noise)
data_dev1['text']=data_dev1['text'].apply(remove_english)
data_dev1['text']=data_dev1['text'].apply(char_gram)

data_dev1.head()

,text,category
0,با<some_space>ال<some_space>لم<some_space>من<s...,MSA
1,هذ<some_space>ذا<some_space>ا <some_space> ا<s...,MSA
2,هذ<some_space>ذا<some_space>ا <some_space> ا<s...,MSA
3,لم<some_space>م <some_space> ي<some_space>يخ<s...,MSA
4,عن<some_space>ند<some_space>دك<some_space>ك <s...,MSA


In [ ]:
data_task1[data_task1.category == 'CAI']

In [ ]:
# THIS IS FOR TASK 2! - to add a feature of whether the tweet is a retweet. 
# Add column to say whether the tweet is a retweet or not - Takes forever
# Use the other dataset 
data["is_retweet"] = 0
for i in range(data.shape[0]):
    if "RT" in data.loc[i, "tweet_text"]:
        data.loc[i, "is_retweet"] = 1
    else:
        data.loc[i, "is_retweet"] = 0

In [253]:
# FINAL DATASET FOR TASK 2 WITH RETWEET COLUMN 
# data.to_csv('final2_new_dataset_TRAIN_tweets_features_wRETWEET.csv')
data=pd.read_csv('final2_new_dataset_TRAIN_tweets_features_wRETWEET.csv')
twitter_data2=twitter_data2.drop(twitter_data2.columns[0], axis=1)
data.head()

,Unnamed: 0,user,tweet_ID,language,user_country,tweet_text,ALE,ALG,ALX,AMM,...,MSA,MUS,RAB,RIY,SAL,SAN,SFX,TRI,TUN,is_retweet
0,0,uJ4DPgC8LZxsFJ6,1094598211673690112,ar,Oman,(أَمْسَيْنَا وَأَمْسَى الْمُلْكُ لِله وَالْحَم...,0.0000,0.0000,0.0,0.0000,...,0.0002,0.9976,0.0,0.0010,0.0001,0.0001,0.0000,0.0000,0.0000,1
1,1,uJ4DPgC8LZxsFJ6,1094598132720193543,ar,Oman,اللهم إنى أعوذ الهم و الحزن و أعوذ العجز و الك...,0.0000,0.9648,0.0,0.0001,...,0.0009,0.0034,0.0,0.0075,0.0025,0.0002,0.0046,0.0015,0.0088,1
2,2,uJ4DPgC8LZxsFJ6,1094597914985459713,ar,Oman,(مَن قال يُمسِي ثلاث مرات أَعوذ بِكَلماتِ اللَ...,0.0000,0.0000,0.0,0.0001,...,0.0205,0.4342,0.0,0.0305,0.0013,0.0016,0.0000,0.0001,0.0000,1
3,3,uJ4DPgC8LZxsFJ6,1094597887185637376,ar,Oman,إله سبحانك إني كنت الظالمين,0.0015,0.0000,0.0,0.0079,...,0.0235,0.0099,0.0,0.0001,0.0002,0.0000,0.0000,0.0000,0.0000,1
4,4,uJ4DPgC8LZxsFJ6,1094597862879580162,ar,Oman,قال ﷺ: صلى عليَّ صلاة واحدة صلى الله عشرا ﷺﷺﷺﷺ...,0.0000,0.0000,0.0,0.0000,...,0.9930,0.0063,0.0,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,1


### testing of different models **STARTS HERE**

In [318]:
data_task1.head()

,text,category
0,هن<some_space>نا<some_space>اك<some_space>ك <s...,MSA
1,لم<some_space>م <some_space> ا<some_space>اس<s...,MSA
2,اس<some_space>ست<some_space>تم<some_space>مر<s...,MSA
3,كم<some_space>م <some_space> ت<some_space>تك<s...,MSA
4,كي<some_space>يف<some_space>ف <some_space> أ<s...,MSA


#### (Vectorization of text for task 1) 

In [339]:
# X_train, X_test, Y_train, Y_test = train_test_split(data_task1['text'], data_task1['category'], test_size=0.2,random_state=10)

count = CountVectorizer(tokenizer = lambda x: x.split("<some_space>"), analyzer="word")
# count = CountVectorizer(min_df=3) this if you'd like to the words that have repeated only 3 times. 
# it has been tried and resulted with a less score for SVM 
X_train = count.fit_transform(data_task1['text'])
X_test = count.transform(data_dev1['text'])
X_train.shape, X_test.shape

((101600, 419092), (5200, 419092))

In [ ]:
# ((101600, 146616), (5200, 146616))

In [283]:
X_train[0],data_task1['text'][0]

(<1x146616 sparse matrix of type '<class 'numpy.int64'>'
 	with 79 stored elements in Compressed Sparse Row format>,
 0    هن<some_space>نا<some_space>اك<some_space>ك <s...
 0    شن<some_space>نا<some_space>اه<some_space>هو<s...
 0    بغ<some_space>غي<some_space>يت<some_space>ت <s...
 Name: text, dtype: object)

In [340]:
Y_train = data_task1['category']
Y_test = data_dev1['category']


In [341]:
# Dummization of the labels
dummies=pd.get_dummies(Y_train,prefix="",prefix_sep="",sparse=True)
labels=dummies.columns
Z=dummies

### SUPPORT VECTOR MACHINE 

In [ ]:
from sklearn.svm import SVC

# Cs = [0.01,0.1,1,10]
# val_error = []

# for C in Cs:

cross_val = cross_val_score(model, X_train, Y_train, cv=5)
val_err=cross_val.mean()
val_error.append(val_err)
print(C,val_err)

In [227]:
from sklearn.svm import SVC

# Pick C=1 as best value for hyperparameter and retrain on whole training set
model=SVC(C=1, kernel='linear')
model.fit(X_train, Y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [228]:
Y_pred = model.predict(X_test)
np.mean(Y_pred==Y_test)

0.5486538461538462

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_pred, Y_test)

In [230]:
scoreSVM=f1_score(Y_test, Y_pred, average='weighted')  
scoreSVM

0.5521264290839432

### Logistic Regression 

In [342]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [343]:
from sklearn.metrics import f1_score

y_pred2 = logreg.predict(X_test)
scorelogreg=f1_score(Y_test, y_pred2, average='weighted')  
scorelogreg
# for char_gram with n = 3, the f-1 score is 0.6642045984948366, second trial = 0.6714308807323026
# for char_gram with n = 5, the f-1 score is 0.5407657629111126
# ------------ 
## ACTUAL SCORES ON DEV CORPUS 26: 
# WITH n=4 for char-level grams (including n=3,2) and word unigrams: F-1 score = 0.581698
# WITH n=3 only and word unigrams: F-1 score = 0.56254
# WITH n=3 including n=2 with word unigrams: F-1 score = 0.56875
# WITH n=4 and word unigrams: F-1 score = 0.56525
# WITH n=4 ONLY: F-1 score = 0.5715
# WITH n=5 for char-level grams (including n=4,3,2) and word unigrams: f1 score = 0.5830




0.5830062191509544

### XGBoost  

In [48]:
from xgboost import XGBClassifier

modelXGB = XGBClassifier()
modelXGB.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
y_predXGB = model.predict(X_test)
scoreXGB=f1_score(Y_test, y_predXGB, average='weighted')  
scorelogreg=f1_score(Y_test, y_pred2, average='weighted')  

In [247]:
print('SVM F1-score: ' + str(scoreSVM))
print('Log Reg F1-score: ' + str(scorelogreg))
print('XGBoost F1-score: ' + str(scoreXGB))

SVM F1-score: 0.5521264290839432
Log Reg F1-score: 0.5625453758463514
